In [10]:
import pandas as pd
import requests
import sys
import logging
import os
import datetime as dt
sys.path.append('../src')
print(sys.path)  # Debug: check if '../src' is in sys.path

from utils.fetch_data import fetch_nwis_data
import utils.duckdb_utils as du
import utils.site_list as sl



['/Users/christophermichaud/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python312.zip', '/Users/christophermichaud/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12', '/Users/christophermichaud/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12/lib-dynload', '', '/Users/christophermichaud/git_projects/ucpo_waterdata/.venv/lib/python3.12/site-packages', '../src', '/Users/christophermichaud/git_projects/ucpo_waterdata/.venv/lib/python3.12/site-packages/setuptools/_vendor', '../src']


In [2]:
# Configure logging ------------------------------------------------
os.makedirs('logs', exist_ok=True)
log_name = 'logs/' + dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log'
logging.basicConfig(filename=log_name,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
# -------------------------------------------------------------------

In [15]:
print(sl.nwis_sites)
sites = ", ".join(sl.nwis_sites)

with du.connect_duckdb() as con:
    result = con.execute("SELECT site_cd from site WHERE agency_cd = 'USGS'").fetchall()
    sites = [row[0] for row in result]
    print(sites)

with du.connect_duckdb() as con:
    result = con.execute("SELECT site_cd, site_nm from site WHERE agency_cd = 'USGS' AND hydro_area_cd = 'GR'").df()
    print(result)
    

['09152500', '09095500', '09106150', '09106485', '09163500', '09306500', '09251000', '09260050', '09260000', '09261000', '09315000', '09302000', '09180000', '09328960', '09147022', '09041395', '09379900']
['09041395', '09095500', '09106150', '09106485', '09147022', '09152500', '09163500', '09180000', '09251000', '09260000', '09260050', '09261000', '09302000', '09306500', '09315000', '09328960', '09379900']
    site_cd                         site_nm
0  09261000     GREEN RIVER NEAR JENSEN, UT
1  09315000  GREEN RIVER AT GREEN RIVER, UT


In [5]:
nwis_rename_map = {
    'site_no': 'site_cd',
    'station_nm': 'site_nm',
    'agency_cd': 'agency_cd',
    'dec_lat_va': 'lat_dd',
    'dec_long_va': 'lon_dd',
    'alt_va': 'elev_ft',
    'site_tp_cd': 'site_type'
    }

bor_rename_map = {
    'site_id': 'site_cd', 'site_metadata.site_name': 'site_nm', 
    'site_metadata.lat': 'lat_dd', 'site_metadata.longi': 'lon_dd', 
    'site_metadata.elevation': 'elev_ft',

}

In [ ]:
nwis_metadata = fetch_nwis_data(
    site=",".join(sites),
    service_code='site')

In [17]:
import requests

def get_series_catalog(site, services=("iv", "dv")):
    
    for service in services:
        url = f"https://waterservices.usgs.gov/nwis/{service}/?sites={site}&parameterCd=all&seriesCatalogOutput=true&format=json"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            return data, service
        except requests.exceptions.HTTPError as e:
            if response.status_code == 400:
                continue  # No data for this service, try next
            else:
                print(f"HTTP error for site {site} ({service}): {e}")
                break
        except Exception as e:
            print(f"Error fetching {service} data for site {site}: {e}")
            break
    return None, None

# Example usage
site = "09315000"  # Example site number
service = get_series_catalog(site)

In [14]:
# Fetching data from USGS NWIS for each site
results = []
for site in sites:
    url = (
        f'https://waterservices.usgs.gov/nwis/dv/?sites={site}'
        f'&parameterCd=all&seriesCatalogOutput=true&format=json'
    )
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()

        time_series_list = data.get("value", {}).get("timeSeries", [])
        for series in time_series_list:
            results.append({
                "site_cd": series.get("sourceInfo", {}).get("siteCode", [{}])[0].get("value"),
                "site_nm": series.get("sourceInfo", {}).get("siteName"),
                "parameter_cd": series.get("variable", {}).get("variableCode", [{}])[0].get("value"),
                "parameter_nm": series.get("variable", {}).get("variableName"),
                "start_date": series.get("variableTimeInterval", {}).get("start"),
                "end_date": series.get("variableTimeInterval", {}).get("end"),
            })
    except Exception as e:
        print(f"Error fetching data for site {site}: {e}")

df = pd.DataFrame(results)



Error fetching data for site 09041395: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09041395&parameterCd=all&seriesCatalogOutput=true&format=json
Error fetching data for site 09095500: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09095500&parameterCd=all&seriesCatalogOutput=true&format=json
Error fetching data for site 09106150: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09106150&parameterCd=all&seriesCatalogOutput=true&format=json
Error fetching data for site 09106485: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09106485&parameterCd=all&seriesCatalogOutput=true&format=json
Error fetching data for site 09147022: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09147022&parameterCd=all&seriesCatalogOutput=true&format=json
Error fetching data for site 09152500: 400 Client Error:  for url: https://waterservices.usgs.gov/nwis/dv/?sites=09152500&par

In [ ]:
nwis_metadata = fetch_nwis_data(
    site=sites,
    service_code='site')

selected = nwis_metadata[list(nwis_rename_map)].rename(columns=nwis_rename_map).reset_index(drop=True)
merged = selected.merge(sl.hydrologic_areas, on='site_cd', how='left')
merged['elev_m'] = (merged['elev_ft'] * 0.3048).round()
merged['lat_dd'] = merged['lat_dd'].round(7)
merged['lon_dd'] = merged['lon_dd'].round(7)
merged['agency_nm'] = 'US Geological Survey'
merged['site_type'] = merged['site_type'].str.replace('ST', 'Stream')
merged['site_type'] = merged['site_type'].str.replace('LK', 'Lake')
merged['source'] = 'NWIS API'
merged['site_dsc'] = None
merged['site_id'] = range(1, len(merged) + 1)
merged['create_ts'] = dt.datetime.now()
merged['update_ts'] = dt.datetime.now()

nwis_metadata_cleaned = merged[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']].sort_values(by='site_cd')


In [5]:
bor_metadata = pd.read_csv("https://www.usbr.gov/uc/water/hydrodata/reservoir_data/meta.csv")
bor_selected = bor_metadata[list(bor_rename_map)].rename(columns=bor_rename_map).reset_index(drop=True)
bor_selected['site_cd'] = bor_selected['site_cd'].astype(str)
bor_filtered = bor_selected[bor_selected['site_cd'].isin(sl.bor_sites)].drop_duplicates(subset='site_cd')

bor_merged = bor_filtered.merge(sl.hydrologic_areas, on='site_cd', how='left')
bor_merged['elev_m'] = (bor_merged['elev_ft'] * 0.3048).round()
bor_merged['lat_dd'] = bor_merged['lat_dd'].round(7)
bor_merged['lon_dd'] = bor_merged['lon_dd'].round(7)
bor_merged['agency_nm'] = 'US Bureau of Reclamation'
bor_merged['agency_cd'] = 'BOR'
bor_merged['site_type'] = "Lake"
bor_merged['source'] = 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/<site_cd>/csv/<parameter_cd>.csv'
bor_merged['site_dsc'] = None
bor_merged['site_id'] = range(18, len(bor_merged) + 18)
bor_merged['create_ts'] = dt.datetime.now()
bor_merged['update_ts'] = dt.datetime.now()

bor_metadata_cleaned = bor_merged[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']]



In [3]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT MAX(site_id) FROM site").fetchone()
    print(f"max site_id from 'site' table: {result[0]}")
    #logging.info(f"Number of rows in 'site' table: {result[0]}")


max site_id from 'site' table: 27


In [4]:
cbrfc_metadata = sl.cbrfc_sites_df.copy()
cbrfc_metadata['site_id'] = range(28, len(cbrfc_metadata) + 28)
cbrfc_metadata['create_ts'] = dt.datetime.now()
cbrfc_metadata['update_ts'] = dt.datetime.now()
cbrfc_metadata_cleaned = cbrfc_metadata[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']]

In [ ]:
print(sl.hydrologic_areas)
ck = pd.read_csv(f"https://www.usbr.gov/uc/water/hydrodata/reservoir_data/{bor_merged['site_cd'][0]}/csv/17.csv")

In [ ]:
du.run_sql_file('../db/schema.sql')

In [5]:
query = """
INSERT INTO site
SELECT * FROM staging_table
"""
print(query)


INSERT INTO site
SELECT * FROM staging_table



In [ ]:
with du.connect_duckdb() as con:
    columns = con.execute("PRAGMA table_info('site')").fetchall()
    print(columns)

In [6]:
with du.connect_duckdb() as con:
        try:
            con.register('staging_table', cbrfc_metadata_cleaned)
            con.execute(query)
            logging.info(f"✅ Successfully executed SQL query: {query}")
        except Exception as e:
            logging.error(f"❌ Error executing SQL file {query}: {e}")
            raise


In [16]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT COUNT(*) FROM site").fetchone()
    print(f"Number of rows in 'site' table: {result[0]}")
    logging.info(f"Number of rows in 'site' table: {result[0]}")


Number of rows in 'site' table: 45


In [7]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT * FROM site").df()

In [21]:
result.to_csv('site_metadata.csv', index=False)